<a href="https://colab.research.google.com/github/MuskanThakur-13/spam-ham/blob/main/spamHamwithoutOversampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from time import perf_counter

In [ ]:
from google.colab import files
data = files.upload()

In [ ]:
data = pd.read_csv('final_data.csv')

In [ ]:
data.head()

In [ ]:
data['label'].value_counts()

In [ ]:
data['label'].value_counts().plot.bar(color = ["g","r"])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['text']).toarray()

In [ ]:
y = np.array(data['label_num'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
import plotly.express as px
from sklearn.decomposition import PCA



pca = PCA(n_components=2)
components = pca.fit_transform(X_train)

fig = px.scatter(components, x = 0, y = 1, color=y_train)
fig.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['text'])

In [ ]:
y = np.array(data['label_num'])
     

In [ ]:
models = {
    "Random Forest": {"model":RandomForestClassifier(), "perf":0},
    "MultinomialNB": {"model":MultinomialNB(), "perf":0},
    "Logistic Regr.": {"model":LogisticRegression(), "perf":0},
    "KNN": {"model":KNeighborsClassifier(), "perf":0},
    "SVM (Linear)": {"model":LinearSVC(), "perf":0},
    "SVM (RBF)": {"model":SVC(), "perf":0}
}

In [ ]:
for name, model in models.items():
    start = perf_counter()
    model['model'].fit(X_train, y_train)
    duration = perf_counter() - start
    duration = round(duration,2)
    model["perf"] = duration
    print(f"{name:20} trained in {duration} sec")
     

In [ ]:
models_acc = []
for name, model in models.items():
    models_acc.append([name, model["model"].score(X_test, y_test),model["perf"]])

In [ ]:
df_acc = pd.DataFrame(models_acc)
df_acc.columns = ['Model', 'Accuracy without scaling (test set)', 'Training time (sec)']
df_acc.sort_values(by = 'Accuracy without scaling (test set)', ascending = False, inplace=True)
df_acc.reset_index(drop = True, inplace=True)
df_acc

In [ ]:
plt.figure(figsize = (15,5))
sns.barplot(x = 'Model', y = 'Accuracy without scaling (test set)', data = df_acc)
plt.title('Accuracy on the test set\n(Without oversampling)', fontsize = 15)
plt.ylim(0.8,1)
plt.show()

In [ ]:
plt.figure(figsize = (15,5))
sns.barplot(x = 'Model', y = 'Training time (sec)', data = df_acc)
plt.title('Training time for each model in sec without oversampling', fontsize = 15)
plt.ylim(0,20)
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import fbeta_score, f1_score
from sklearn.metrics import roc_auc_score

In [ ]:
model_stats = []
for name, model in models.items():
    
    y_pred = model["model"].predict(X_test)
    model_stats.append([name, model["model"].score(X_test, y_test),model["perf"],precision_score(y_test, y_pred), recall_score(y_test, y_pred),f1_score(y_test, y_pred),fbeta_score(y_test, y_pred, beta = 2), roc_auc_score(y_test, y_pred)])
    print(f'{name} appended to stats')        #only to track the progress

In [ ]:
df_stats = pd.DataFrame(model_stats)
df_stats.columns = ['Model', 'Accuracy without scaling (test set)', 'Training time (sec)', 'Precision', 'Recall','F1 score', 'F2 score', 'ROC-AUC score']
df_stats.sort_values(by = 'Accuracy without scaling (test set)', ascending = False, inplace=True)
df_stats.reset_index(drop = True, inplace=True)
df_stats